In [2]:
import django_initializer
from backend.pymongo.mongodb import db
from datetime import datetime
import arrow

In [5]:
start_time = "2022-06-15"
end_time = "2022-06-15"
new_start_time = arrow.get(start_time).replace(hour=0, minute=0, second=0).datetime
display(new_start_time)
new_end_time = arrow.get(start_time).replace(hour=23, minute=59, second=59).datetime
display(new_end_time)

datetime.datetime(2022, 6, 15, 0, 0, tzinfo=tzutc())

datetime.datetime(2022, 6, 15, 23, 59, 59, tzinfo=tzutc())

In [23]:
cursor=db.api_campaign.aggregate([
    {
        "$match":{
            "start_at":{
                "$gte": new_start_time
            },
            "end_at": {
                "$lte": new_end_time
            }
        }
    },
    {
        "$lookup": {
            "from": "api_campaign_comment",
            "as": "campaign_comment",
            'let': {'id': "$id" },
            "pipeline":[
                {"$match":{
                    '$expr': { '$eq': ["$$id", "$campaign_id"] },
                    "id":{"$ne":None}}
                }
            ]
        },
    },
    {
        "$lookup": {
            "from": "api_campaign_product",
            "as": "campaign_product_sold", 
            'let': {'id': "$id" },
            "pipeline":[
                {
                    "$match":{
                        '$expr': { '$eq': ["$$id", "$campaign_id"] },
                        "id":{"$ne":None},
                        "qty_sold": {
                            "$gt": 0
                        }
                    }
                }
            ]
        },
    },
    {
        "$lookup": {
            "from": "api_campaign_product",
            "as": "campaign_product_unsold", 
            'let': {'id': "$id" },
            "pipeline":[
                {
                    "$match":{
                        '$expr': { '$eq': ["$$id", "$campaign_id"] },
                        "id":{"$ne":None},
                        "qty_sold": {
                            "$eq": 0
                        }
                    }
                }
            ]
        },
    },
    {
        "$lookup": {
            "from": "api_campaign_product",
            "as": "campaign_product_total_item", 
            'let': {'id': "$id" },
            "pipeline":[
                {
                    "$match":{
                        '$expr': { '$eq': ["$$id", "$campaign_id"] },
                        "id":{"$ne":None},
                    }
                }
            ]
        },
    },
    {
        "$lookup": {
            "from": "api_order",
            "as": "orders", 
            'let': {'id': "$id" },
            "pipeline":[
                {
                    "$match":{
                        '$expr': { '$eq': ["$$id", "$campaign_id"] },
                        "id":{"$ne":None},
                    }
                }
            ]
        },
    },
    {
        "$project":{
            "_id":0,
            "campaign_id":"$id",
            "post_comment":{"$size":"$campaign_comment.id"},
            "no_of_items_sold":{"$size": "$campaign_product_sold.id"},
            "no_of_items_unsold":{"$size": "$campaign_product_unsold.id"},
            "total_no_of_items":{"$size": "$campaign_product_total_item.id"},
            "total_inventories":{"$sum": "$campaign_product_total_item.qty_for_sale"},
            "average_order_value":{"$avg": "$orders.total"},
            "total_amount": {"$sum": "$orders.total"},
            "total_no_of_orders": {"$size": "$orders.total"},
            
        }
    }
])
l = list(cursor)

In [24]:
l

[{'campaign_id': 495,
  'post_comment': 2,
  'no_of_items_sold': 1,
  'no_of_items_unsold': 2,
  'total_no_of_items': 3,
  'total_inventories': 1000,
  'average_order_value': None,
  'total_amount': 0,
  'total_no_of_orders': 0},
 {'campaign_id': 496,
  'post_comment': 4,
  'no_of_items_sold': 4,
  'no_of_items_unsold': 2,
  'total_no_of_items': 6,
  'total_inventories': 1050,
  'average_order_value': 127.92,
  'total_amount': 127.92,
  'total_no_of_orders': 1},
 {'campaign_id': 497,
  'post_comment': 14,
  'no_of_items_sold': 1,
  'no_of_items_unsold': 1,
  'total_no_of_items': 2,
  'total_inventories': 115,
  'average_order_value': 12.5,
  'total_amount': 25.0,
  'total_no_of_orders': 2},
 {'campaign_id': 499,
  'post_comment': 13,
  'no_of_items_sold': 3,
  'no_of_items_unsold': 1,
  'total_no_of_items': 4,
  'total_inventories': 400,
  'average_order_value': 8870.0,
  'total_amount': 8870.0,
  'total_no_of_orders': 1},
 {'campaign_id': 501,
  'post_comment': 16,
  'no_of_items_sold